In [ ]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import networkx as nx
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
import re
from nltk.stem import WordNetLemmatizer
SET_WINDOW_LENGTH=2
import matplotlib.pyplot as plt
from nltk.tokenize import WordPunctTokenizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [ ]:
import pandas as pd
df1=pd.read_csv('drive/MyDrive/Chandni Muskan/Explainability for CAMS/CNN_LSTM_intepretationIG.csv', usecols= ['top_tokens','Interpretations','label'])
print(df1)
# Reset index after drop
df=df1.dropna().reset_index(drop=True)
print(df)

                                       Interpretations  \
0    not productive, worthless, hate, bored , issue...   
1    feeling trapped, paranoid, barely talks to me,...   
2                                                 ugly   
3    abuse suffered, hard person to love, scars, cy...   
4    wasting money, sulking, failure, disappointmen...   
..                                                 ...   
365                                                NaN   
366  learn, sad, assure, worse, logic, leave, const...   
367                  gotten worse, tried so hard, pain   
368  anxiety, friends are disappearing, hit really ...   
369  ending the relationship, donât speak to my m...   

                                            top_tokens  label  
0    ['constantly', 'everyday', 'looking', 'therapi...      1  
1    ['hours', 'everyday', 'hes', 'wrong', 'took', ...      5  
2            ['health', 'believe', None, 'im', 'dont']      1  
3    ['wouldnt', 'us', 'gets', 'wants', 'taking

In [ ]:
#PRE-PROCESSING
def remove_punctuation(test_str):
  test_str = test_str.translate(str.maketrans(' ', ' ', string.punctuation))
  return test_str
def remove_stopwords(list_of_words):
  stop_words = set(stopwords.words('english'))
  texts=[t for t in list_of_words if t not in stop_words]
  return texts
def lemmatize(texts):
  lemmatizer = WordNetLemmatizer()
  new_texts=[]
  for each_word in texts:
    new_texts.append(lemmatizer.lemmatize(each_word))
  return new_texts
def remove_noise(texts):
  new_texts=[]
  for each_word in texts:
    if len(each_word)>1:
      new_texts.append(each_word )
  return new_texts
def preprocess(texts):
  texts=remove_punctuation(texts)
  texts=texts.lower()
  texts=WordPunctTokenizer().tokenize(texts)
  texts=remove_stopwords(texts)
  texts=lemmatize(texts)
  texts=remove_noise(texts)
  texts=' '.join(texts)
  return texts


In [ ]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize

In [ ]:
def taggeddata(data):
  tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(data)]

In [ ]:
import gensim
from gensim.models import Word2Vec
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
tokenized_doc=[]
tokenized_doc=list(df['top_tokens'])


tagged_data = [TaggedDocument(d, [i]) for i, d in enumerate(tokenized_doc)]

model1 = Doc2Vec(tagged_data, size = 100, window = 100, min_count = 1, workers = 4)
#model1 = Word2Vec(tagged_data,  size = 100, window = 5, min_count = 1)

tokenized_doc=[]
for i in range(0, len(list(df['Interpretations']))):
  values=df['Interpretations'][i]
  values=str(values).split(',')
  tokenized_doc.append(values)

tagged_data1 = [TaggedDocument(d, [i]) for i, d in enumerate(tokenized_doc)]

model2 = Doc2Vec(tagged_data1, size = 100, window = 100, min_count = 1, workers = 4)
#model2 = Word2Vec(tagged_data, size = 100, window = 5, min_count = 1)
#model = Doc2Vec(tagged_data, size = 100, window = 300, min_count = 1, workers = 4)

#! pip install testfixtures
#from gensim.test.test_doc2vec import ConcatenatedDoc2Vec
#new_model = ConcatenatedDoc2Vec((model1, model2))

/usr/local/lib/python3.7/dist-packages/gensim/models/doc2vec.py:570: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


In [ ]:
import numpy as np
from scipy.spatial.distance import cosine
def cosine_similarity(vector_1: np.array, vector_2: np.array) -> float:
    """
    :param vector_1: vector of the first document
    :param vector_2: vector of the second document
    :return: cosine distance between the two documents
    """

    # calculate the cosine distance between two vectors
    if np.all((vector_1 == 0)) or np.all((vector_2 == 0)):
      return 0

    # return 1 - cosine_distance to indicate similarity
    cosine_distance = cosine(vector_1, vector_2)
    return 1 - cosine_distance

In [ ]:
# import numpy as np
# ! pip install rouge-score
# from rouge_score import rouge_scorer

In [ ]:
# !pip install textdistance
# !pip install "textdistance[benchmark]"
# !pip install "textdistance[extras]"

In [ ]:
import gensim
from scipy import spatial
import gensim.downloader as api

model = api.load("glove-wiki-gigaword-50") #choose from multiple models https://github.com/RaRe-Technologies/gensim-data
calculate=0

def preprocess(s):
    return [i.lower() for i in s.split()]

def get_vector(s):
    return np.sum(np.array([model[i] for i in preprocess(s)]), axis=0)

[==================================================] 100.0% 66.0/66.0MB downloaded


# Results

In [ ]:
# import textdistance
# #model= Doc2Vec.load("d2v.model")
# text1=[]
# text2=[]
# text0=[]
# calculate=0
# rouge=0
# scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)

# hamming=0
# word2vec=0
# for each_row in range(0, len(df.index)):
#   doc1 = df['top_tokens'][each_row]
#   doc1=doc1.lstrip('[').rstrip(']')
#   doc1=doc1.split(',')
#   text_doc=''
#   for eachword in doc1:
#     eachword=eachword.strip()
#     eachword=eachword.lstrip("'").rstrip("'")
    
#     text_doc=text_doc+ eachword+' '
# #  doc1=''.join(lst_doc1)
# #  print(type(doc1))
  
#   text_doc=preprocess(text_doc)
#   text1.append(text_doc)
#   doc2=df['Interpretations'][each_row]



#   vector1 = model1.infer_vector(text_doc)
#   vector2 = model2.infer_vector(str(doc2))

#   cosine_sim=cosine_similarity(vector1, vector2)
#   calculate=calculate+ abs(cosine_sim)

#   cosine_sim1=model.wmdistance(text_doc, str(doc2))
#   print(cosine_sim1)
#   word2vec=word2vec + abs(cosine_sim1)

#   scores = scorer.score(str(text_doc), str(doc2))
#   print(scores)
#   #rouge=rouge+scores


#   hamming=hamming + textdistance.hamming.distance(text_doc, str(doc2))

# print(calculate/370.0) #Doc2vec
# print(hamming/370.0) # Hamming
# print(word2vec/370.0) # word2vec
# print(rouge/370.0) # word2vec
# print(word2vec)

## 1. Using pretrained word2vec

---



In [ ]:
def get_fun(word):
  try:
    return get_vector(word)
  except:
    return 4


In [ ]:
import math

calculate=0
flag=0
dist0=dist1=dist2=dist3=dist4=dist5=0.0
count0=count1=count2=count3=count4=count5=0
text1=[]
for each_row in range(0, len(df.index)):
  doc1 = df['top_tokens'][each_row]
  doc1=doc1.lstrip('[').rstrip(']')
  doc1=doc1.split(',')
  text_doc=''
  for eachword in doc1:
    eachword=eachword.strip()
    eachword=eachword.lstrip("'").rstrip("'")
    
    text_doc=text_doc+ eachword+' '
  
#  doc1=''.join(lst_doc1)
#  print(type(doc1))
  
  text_doc=preprocess(text_doc)
  text1.append(text_doc)

  doc2=df['Interpretations'][each_row]

  test=str(doc2)
  doc2=preprocess(str(doc2))


  text_doc2=[]
  for eachword in doc2:

#   eachword=eachword.lstrip(" ").rstrip(" ")
    eachword=eachword.replace(",","")

    if not eachword.isdigit():
      text_doc2.append(str(eachword))
#  doc2=''.join(doc2)
  abc=[]
  vector1 = np.mean([get_fun(word) for word in text_doc], axis=0)
  for word in text_doc2:
    get=get_fun(word)

    if get is not 4:
      abc.append(get)

  vector2 = np.mean(abc, axis=0)

  if(test!='nan'):
  #  print("NAN")
# else:
    cosine_sim=cosine_similarity(vector1, vector2)
    x=abs(cosine_sim)
    if (math.isnan(cosine_sim)):
      calculate=calculate+ abs(cosine_sim)
      flag=flag+1
      x=0.0

 #   x=abs(cosine_sim)

    if(df['label'][each_row])==0:
      dist0=dist0+x
      count0=count0+1
    elif (df['label'][each_row])==1:
      dist1=dist1+x
      count1=count1+1
    elif (df['label'][each_row])==2:
      dist2=dist2+x
      count2=count2+1
    elif (df['label'][each_row])==3: 
      dist3=dist3+x
      count3=count3+1
    elif (df['label'][each_row])==4: 
      dist4=dist4+x
      count4=count4+1
    else:
      dist5=dist5+x
      count5=count5+1
print(calculate/(float)(flag))
print("Cosine between interpretations and top keywords for class 0 is ", dist0/count0)  
print("Cosine between interpretations and top keywords for class 1 is ", dist1/count1)  
print("Cosine between interpretations and top keywords for class 2 is ", dist2/count2) 
print("Cosine between interpretations and top keywords for class 3 is ", dist3/count3)
print("Cosine between interpretations and top keywords for class 4 is ", dist4/count4)  
print("Cosine between interpretations and top keywords for class 5 is ", dist5/count5)  


nan
Cosine between interpretations and top keywords for class 0 is  0.7283725556917489
Cosine between interpretations and top keywords for class 1 is  0.7893811890057155
Cosine between interpretations and top keywords for class 2 is  0.851162555317084
Cosine between interpretations and top keywords for class 3 is  0.6900965016941691
Cosine between interpretations and top keywords for class 4 is  0.8702237370249989
Cosine between interpretations and top keywords for class 5 is  0.8152743490493816


/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:163: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = asanyarray(a)
/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3441: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [ ]:

df2=df.groupby('label')
for eachdf in df2:
    calculate=0
    flag=0
    text1=[]
    eachdf=eachdf.reset_index()
    for each_row in range(0, len(eachdf)):
      doc1 = eachdf['top_tokens'][each_row]
      doc1=doc1.lstrip('[').rstrip(']')
      doc1=doc1.split(',')
      text_doc=''
      for eachword in doc1:
        eachword=eachword.strip()
        eachword=eachword.lstrip("'").rstrip("'")
        
        text_doc=text_doc+ eachword+' '
      
    #  doc1=''.join(lst_doc1)
    #  print(type(doc1))
      
      text_doc=preprocess(text_doc)
      text1.append(text_doc)

      doc2=eachdf['Interpretations'][each_row]

      test=str(doc2)
      doc2=preprocess(str(doc2))
    
    
      text_doc2=[]
      for eachword in doc2:

    #   eachword=eachword.lstrip(" ").rstrip(" ")
        eachword=eachword.replace(",","")

        if not eachword.isdigit():
          text_doc2.append(str(eachword))
    #  doc2=''.join(doc2)
      abc=[]
      vector1 = np.mean([get_fun(word) for word in text_doc], axis=0)
      for word in text_doc2:
        get=get_fun(word)
    
        if get is not 4:
          abc.append(get)

      vector2 = np.mean(abc, axis=0)

      if(test!='nan'):
      #  print("NAN")
    # else:
        cosine_sim=cosine_similarity(vector1, vector2)
        if (str(cosine_sim)!='nan'):
          calculate=calculate+ abs(cosine_sim)
          flag=flag+1
    print(calculate/(float)(flag))

AttributeError: ignored

In [ ]:
print(calculate/(float)(flag))
print(flag)

In [ ]:
import gensim.downloader as api

# Load the pre-trained model
model = api.load('fasttext-wiki-news-subwords-300')
calculate=0

def preprocess(s):
    return [i.lower() for i in s.split()]

def get_vector(s):
    return np.sum(np.array([model[i] for i in preprocess(s)]), axis=0)

In [ ]:

text1=[]
flag=0

#df_list=df.groupby(['label'])
#print(df_list)
df2=df.groupby('label')
for eachdf in df2:
    calculate=0
    flag=0
    text1=[]
    for each_row in range(0, len(eachdf)):
      doc1 = df['top_tokens'][each_row]
      doc1=doc1.lstrip('[').rstrip(']')
      doc1=doc1.split(',')
      text_doc=''
      for eachword in doc1:
        eachword=eachword.strip()
        eachword=eachword.lstrip("'").rstrip("'")
        
        text_doc=text_doc+ eachword+' '
      
    #  doc1=''.join(lst_doc1)
    #  print(type(doc1))
      
      text_doc=preprocess(text_doc)
      text1.append(text_doc)

      doc2=df['Interpretations'][each_row]

      test=str(doc2)
      doc2=preprocess(str(doc2))
    
    
      text_doc2=[]
      for eachword in doc2:

    #   eachword=eachword.lstrip(" ").rstrip(" ")
        eachword=eachword.replace(",","")

        if not eachword.isdigit():
          text_doc2.append(str(eachword))
    #  doc2=''.join(doc2)
      abc=[]
      vector1 = np.mean([get_fun(word) for word in text_doc], axis=0)
      vector1=vector1.reshape(1,-1)
      for word in text_doc2:
        get=get_fun(word)
    
        if get is not 4:
          abc.append(get)

      vector2 = np.mean(abc, axis=0)
      vector2=vector2.reshape(1,-1)
      print(vector1.shape)
      print(vector2.shape)
      cosine_sim=cosine_similarity(vector1, vector2)
      if (str(cosine_sim)!='nan'):
          calculate=calculate+ abs(cosine_sim)
          flag=flag+1
    print(calculate/(float)(flag))

In [ ]:
#GUNJAN

## 2.Jaccard Similarity

---



In [ ]:
#!pip install textdistance
#!pip install "textdistance[benchmark]"
#!pip install "textdistance[extras]"


In [ ]:
# import textdistance

# print('Hamming distance1',textdistance.hamming('test', 'text'))


# print ('Hamming distance2',textdistance.hamming.distance('test', 'text'))


# print ('Hamming distance3',textdistance.hamming.similarity('test', 'text'))


# print ('Hamming distance4',textdistance.hamming.normalized_distance('test', 'text'))


# print ('Hamming distance5',textdistance.hamming.normalized_similarity('test', 'text'))


# print ('Hamming distance6',textdistance.Hamming(qval=2).distance('test', 'text'))


In [ ]:
# import textdistance
# from sklearn.metrics.pairwise import cosine_similarity
# import spacy
# nlp = spacy.load("en_core_web_sm")

# def text_processing(sentence):
#     """
#     Lemmatize, lowercase, remove numbers and stop words
    
#     Args:
#       sentence: The sentence we want to process.
    
#     Returns:
#       A list of processed words
#     """
#     sentence = [token.lemma_.lower()
#                 for token in nlp(sentence) 
#                 if token.is_alpha and not token.is_stop]
    
#     return sentence

# def jaccard_sim(row):
#     # Text Processing
#     sentence1 = text_processing(row['sentence1'])
#     sentence2 = text_processing(row['sentence2'])
    
#     # Jaccard similarity
#     return textdistance.jaccard.normalized_similarity(sentence1, sentence2)






## 3. Word Mover Distance

---



In [ ]:
# import gensim.downloader as api

# # Load the pre-trained model
# model = api.load('fasttext-wiki-news-subwords-300')

# def word_movers_distance(row):
#     # Text Processing
#     sentence1 = text_processing(row['sentence1'])
#     sentence2 = text_processing(row['sentence2'])
    
#     # Negative Word Movers Distance
#     return model.wmdistance(sentence1, sentence2)


In [ ]:
word_movers_distance(row)